<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2022/ex14noteB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex14noteB

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# 正規分布の確率密度関数
from scipy.stats import norm
from scipy.stats import multivariate_normal

----
## 統計的パターン認識入門
----

前回登場した密度推定は，教師なし学習の一種とみなせるのでした．
この密度推定を教師あり学習の識別の問題へと応用することを考えます．



---
### 統計的パターン認識とは






識別に関する研究分野や手法のうち，特に画像や音声のような「パターン情報」を対象とするものを，**パターン認識**（pattern recognition）といいます．今回扱う**統計的パターン認識**(statistical pattern recognition)は，その中でも，データ $\mathbf{x}$ やそのクラスラベル $y$ を確率変数とみなし，写像 $\mathbf{x} \rightarrow y$ を確率的なものとして扱う方法です．



画像の画素値を入力すると，それらを「ネコ」「イヌ」「カピバラ」の3クラスに分類する識別問題を考えてみましょう．入力を連続型確率変数として $\mathbf{x}$ と表し，3つのクラスを離散型確率変数 $y$ で表します
（$y \in \{ \mbox{ネコ}, \mbox{イヌ}, \mbox{カピバラ} \}$）．
このとき，$\mathbf{x}$の値が与えられたときの3つの $y$ の確率，つまり条件付き確率 $p(y|\mathbf{x})$ の値が
$$
p(\mbox{ネコ}|\mathbf{x}) = 0.2,\quad
p(\mbox{イヌ}|\mathbf{x}) = 0.75,\quad
p(\mbox{カピバラ}|\mathbf{x}) = 0.05
$$
のように求められれば，「この確率が最大なのは「イヌ」なので，$\mathbf{x}$は「イヌ」に識別する」といったことが可能になります．
これが，統計的パターン認識の基本的な考え方です．

ここで登場した条件付き確率 $p(y|\mathbf{x})$ は，「クラス事後確率」と呼ばれます．詳しくは説明しませんが，ロジスティック回帰は，このクラス事後確率をモデル化して，$\mathbf{x}$はそのクラス事後確率が最大のクラスに識別する，つまり
$$
(\mbox{予測クラス}) = \arg\max_y p(y|\mathbf{x})
$$
というルールを用いる統計的パターン認識の手法となっています．
一方，統計的パターン認識の手法の中には，ロジスティック回帰のように事後確率を直接モデル化する代わりに，**ベイズの公式**
$$
p(\mathbf{x},y) = p(\mathbf{x}|y)p(y) = p(y|\mathbf{x})p(\mathbf{x})
$$
を使って
$$
\arg\max_y p(y|\mathbf{x}) = \arg\max_y \frac{p(\mathbf{x}|y)p(y)}{p(\mathbf{x})} = \arg\max_y p(\mathbf{x}|y)p(y)
$$
と変形できることを利用して，$p(\mathbf{x}|y)$ と $p(y)$ を計算することで識別をする手法もあります（注）.


<span style="font-size: 75%">
※注: この式の最後の変形では，$p(\mathbf{x})$ が $y$ によらないことを利用してこれを無視しています．クラス事後確率を直接モデル化するやり方は「識別モデル」(discriminative model)による手法，$p(\mathbf{x}|y)$ と $p(y)$ から求める手法は「生成モデル」(generative model)による手法と呼ばれます．
</span>

以下では，$p(\mathbf{x}|y)$ と $p(y)$ を計算することで識別をする方法について説明します．こちらの方法では，それぞれ，学習データから次のようにして定めるのが一般的です．

<dl>
<dt>$p(y)$</dt>
<dd>どんな$\mathbf{x}$が来るか知る前の時点でのクラス $y$ の出現確率．「クラス事前確率」と呼ぶ．学習データ中の各クラスの出現頻度をそのまま使ったり，単純に1/クラス数の値を使ったりする．</dd>
<dt>$p(\mathbf{x}|y)$</dt>
<dd>
クラス$y$に所属するデータ$\mathbf{x}$の分布．学習データをクラス毎に分けて，それぞれで密度推定を行うことで求められる．
</dd>
</dl>

---
### ［実験］2次元3クラスのデータを識別してみる


以下のようなデータを対象にして，$p(\mathbf{x}|y)$ を2次元正規分布でモデル化する方法で識別する実験をやってみましょう.


In [ ]:
# 2次元3クラスのデータ
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/2dim3class2.csv', header=0)
X = df.loc[:, ['x1', 'x2']].to_numpy()
N, D = X.shape
lab = df['label'].to_numpy()
K = 3

# 散布図を描く
fig, ax = plt.subplots(facecolor="white", figsize=(8, 8))
ax.scatter(X[lab==0, 0], X[lab==0, 1], label='class0')
ax.scatter(X[lab==1, 0], X[lab==1, 1], label='class1')
ax.scatter(X[lab==2, 0], X[lab==2, 1], label='class2')
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_aspect('equal')
ax.legend()
plt.show()

次のセルを実行すると，$p(\mathbf{x}|y)$ およびクラス事後確率 $p(y|\mathbf{x})$ それぞれの推定結果を可視化します．

In [ ]:
# 事前確率を推定
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(lab==k)/N
print('### 事前確率')
print('p(y) = ', py)

# p(x|y) を正規分布でモデル化（平均，共分散行列の推定）
mu = np.empty((K, D))
cov = np.empty((K, D, D))
for k in range(K):
    XX = X[lab==k, :]
    mu[k] = np.mean(XX, axis=0)
    cov[k] = (XX - mu[k]).T @ (XX - mu[k]) / XX.shape[0]
    print(f'### class {k} の正規分布のパラメータ')
    print(mu[k])
    print(cov[k])

# グラフ描画用のグリッドデータの作成
xmin, xmax = -5, 5
ymin, ymax = -5, 5
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))

# 事後確率の推定
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    p[k, :] = py[k] * multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k]).reshape(-1)
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(12, 8))

# Gaussian を当てはめた結果
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(X[lab==k, 0], X[lab==k, 1])
    ax0.contour(x_mesh, y_mesh, multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k]))
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(X[lab==k, 0], X[lab==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

左図は，$p(\mathbf{x}|\textrm{class0}), p(\mathbf{x}|\textrm{class1}), p(\mathbf{x}|\textrm{class2})$ をそれぞれ2次元正規分布でモデル化し，そのパラメータを学習データから推定した結果を可視化しています．
一方，右図は，$p(\mathbf{x}|y)$ と $p(y)$ から求めたクラス事後確率 $p(y|\mathbf{x})$ を可視化しています．青，オレンジ，緑の色が濃いところは，それぞれ $p(\textrm{class0}|\mathbf{x}), p(\textrm{class1}|\mathbf{x}), p(\textrm{class2}|\mathbf{x})$ の値が大きいところです．
学習データの分布を反映して，うまくクラス間の境界が作られていることがわかります．